In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import accuracy_score
import datetime

In [ ]:
df = pd.read_csv('dataset_finalv3.csv').dropna(subset=['feedback_rating'])
df = df.query('feedback_rating > 0')
df.reset_index(inplace=True)
df.drop(columns='index',inplace=True)

In [ ]:
pd.set_option('display.max_columns', 999)

In [ ]:
#transformando a coluna da data de correção em date time
df.start = pd.to_datetime(df.start)


In [ ]:
#criando uma coluna que diz o ano da correção
df['year'] = [df.loc[i].start.year for i in range(len(df))]


In [ ]:
#criando outro dataframe com dados apenas de 2020 pra cima
df2020 = df.query('year >= 2020')
df2020.reset_index(inplace=True)
df2020.drop(columns='index',inplace=True)

C:\Users\luizf\AppData\Local\Temp\ipykernel_9680\1715257305.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2020.drop(columns='index',inplace=True)


In [ ]:
df2020.dropna(inplace=True)
df2020.reset_index(inplace=True)
df2020.drop(columns=['index'],inplace=True)

C:\Users\luizf\AppData\Local\Temp\ipykernel_9680\3482070822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2020.dropna(inplace=True)
C:\Users\luizf\AppData\Local\Temp\ipykernel_9680\3482070822.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2020.drop(columns=['index'],inplace=True)


In [ ]:
#instalando a biblioteca que fornece os encoders
!pip install category_encoders 

In [ ]:
train = df2020[(df2020.start < datetime.datetime(2021,11,1))].drop(columns=['start','year','oficial_feedback_revis'])
test = df2020[~(df2020.start < datetime.datetime(2021,11,1))].drop(columns=['start','year','oficial_feedback_revis'])

In [ ]:
#aplicando o encoder nas colunas categóricas

import category_encoders as ce 
# parâmetro cols identifica as colunas categóricas
js_treino = ce.JamesSteinEncoder(cols=['tipo_cliente','formato_redacao', 'dia_semana',
                                       'mes_ano','dia_mes','theme_id_essay',
                                       ], model='beta')



treino = js_treino.fit_transform(train.drop(columns='feedback_rating'), train.feedback_rating)

teste = js_treino.transform(test.drop(columns='feedback_rating'))


treino = pd.concat([treino,train.feedback_rating],axis=1)

teste = pd.concat([teste,test.feedback_rating],axis=1)

In [ ]:
X_train2 = treino.drop(columns='feedback_rating')
y_train2 = treino.feedback_rating

X_test2020s = teste.drop(columns='feedback_rating')
y_test2020s = teste.feedback_rating

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 7)

X_nof2020sm, y_nof2020sm = sm.fit_resample(X_train2, y_train2)

In [ ]:
gb = GradientBoostingRegressor(random_state=7)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_absolute_error
results = {}
for i in range(len(X_nof2020sm.columns),1,-1):
  print(f'{i} feature(s)')
  selector = RFE(gb,n_features_to_select=i,verbose = 1)
  selector = selector.fit(X_nof2020sm, y_nof2020sm)
  ypred_gb = selector.predict(X_test2020s)
  results[i] = mean_absolute_error(y_test2020s,ypred_gb)

46 feature(s)
45 feature(s)
Fitting estimator with 46 features.
44 feature(s)
Fitting estimator with 46 features.
Fitting estimator with 45 features.
43 feature(s)
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
42 feature(s)
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
41 feature(s)
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
40 feature(s)
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
39 feature(s)
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 4

Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
24 feature(s)
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estima

Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
15 feature(s)
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estima

Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 fe

Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
3 feature(s)
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator wi

In [ ]:
results

{46: 0.9123230152026195,
 45: 0.9122297976750092,
 44: 0.9122428373521879,
 43: 0.912227577186918,
 42: 0.9133352035415123,
 41: 0.9133352035415123,
 40: 0.9123860450051546,
 39: 0.9125781567231234,
 38: 0.9125781567231234,
 37: 0.9122175791361138,
 36: 0.9119433511228077,
 35: 0.9118875799608117,
 34: 0.9143014837355665,
 33: 0.9141891502305253,
 32: 0.9152455608351938,
 31: 0.9152455608351941,
 30: 0.9142954247217466,
 29: 0.9138818351983906,
 28: 0.912716706082013,
 27: 0.9134552738798813,
 26: 0.9111030750149166,
 25: 0.9129620215920281,
 24: 0.9116647394777074,
 23: 0.9135680007472939,
 22: 0.9140144504457092,
 21: 0.9118352432588422,
 20: 0.9086143312143968,
 19: 0.910100674054571,
 18: 0.9120655341157573,
 17: 0.9136118324842282,
 16: 0.9109206562488398,
 15: 0.9106012638389261,
 14: 0.9317359562788212,
 13: 0.9323423739194244,
 12: 0.9372259380326508,
 11: 0.9402279580766749,
 10: 0.9326943532867062,
 9: 0.9341099960141835,
 8: 0.9435323132347747,
 7: 0.9492363207334493,
 6: 0.

In [ ]:
selector2 = RFE(gb,n_features_to_select=20,verbose = 1)
selector2 = selector2.fit(X_nof2020sm, y_nof2020sm)

Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.


In [ ]:
colunas = []
for i in range(len(X_nof2020sm.columns)):
  if selector2.support_[i] == True:
    print(f'{X_nof2020sm.columns[i]}:{selector2.support_[i]}')
    colunas.append(X_nof2020sm.columns[i])

tamanho_comentario_geral:True
proporcao_negativa:True
qtd_comentarios_positivos:True
qtd_correcoes_anteriores:True
theme_id_essay:True
competencia1:True
competencia2:True
competencia3:True
nota_final:True
media_temporal_aluno:True
media_temporal_corretor:True
std_temporal_aluno:True
mediana_temporal_aluno:True
mediana_temporal_corretor:True
score_medio_temporal_aluno:True
score_std_temporal_aluno:True
feedbacks_positivos_aluno:True
feedbacks_negativos_aluno:True
proporcao_feedback_negativo_aluno:True
proporcao_feedback_negativo_corretor:True


In [ ]:
colunas

['tamanho_comentario_geral',
 'proporcao_negativa',
 'qtd_comentarios_positivos',
 'qtd_correcoes_anteriores',
 'theme_id_essay',
 'competencia1',
 'competencia2',
 'competencia3',
 'nota_final',
 'media_temporal_aluno',
 'media_temporal_corretor',
 'std_temporal_aluno',
 'mediana_temporal_aluno',
 'mediana_temporal_corretor',
 'score_medio_temporal_aluno',
 'score_std_temporal_aluno',
 'feedbacks_positivos_aluno',
 'feedbacks_negativos_aluno',
 'proporcao_feedback_negativo_aluno',
 'proporcao_feedback_negativo_corretor']

In [ ]:
len(colunas)

20